# Connecting to DB2 Mainframe Server Using JayDeBeApi

**GOAL:** Execute a query to run against mainframe DB2 server and return the results back as a pandas dataframe.

To work with mainframe DB2 with 64-bit Python, unfortunately, we must use a specific Java runtime environment version (JRE) that supports the timestamps used in the mainframe tables.  Otherwise, you will get a Java [error](https://stackoverflow.com/questions/36354899/unsupportedcharsetexception-cp1027-with-db2-jdbc-driver).  We also need to have JDBC driver (.jar).

1. Unzip jre.zip file.  Set JAVA_HOME to the path where you unzipped the jre.zip file
2. Add the path to the db2jcc4.jar file to the connect() method or add it to the CLASSPATH environment variable

The db2jcc4.jar file can be downloaded from IBM's [site](https://www.ibm.com/support/pages/db2-jdbc-driver-versions-and-downloads).  However, the file will be compressed in a .tar file which Windows does not know how to unzip.  To unzip the .tar file, you will need to download or install the 7zip software.  Mainframe DB2 z/OS is currently version 10.5.

**UPDATE:** There is [issue](https://www.ibm.com/support/pages/db2-alias-name-behaviour-changes-latest-drivers-versions) with version JDBC drivers >= 4 where column name alias are not being utilized, but return the actual column name instead.  The fix is to append ```:useJDBC4ColumnNameAndLabelSemantics=false;``` to the end of the connection string.

In [ ]:
from getpass import getpass
import jaydebeapi as jdba
import pandas as pd

#### Confirm ```JAVA_HOME``` was set:

In [ ]:
!set JAVA_HOME

#### Provide database credentials

In [ ]:
user = input("Enter your DB2 user id: ")
pwd   = getpass("Enter your DB2 password: ")

#### Server details

In [ ]:
host = 'some.host'
port = '50000'
database = 'your_database'

#### SQL query to obtain current date and time

In [ ]:
sql ="""
SELECT
    CURRENT TIMESTAMP as DATETIME_NOW
FROM
    SYSIBM.SYSDUMMY1
"""

#### Usually best practice to use `with` statement when making a database connection so that we can ensure database connections are closed

In [ ]:
with jdba.connect('com.ibm.db2.jcc.DB2Driver',
    f'jdbc:db2://{host}:{port}/{database}:useJDBC4ColumnNameAndLabelSemantics=false;',
    [user, pwd],
    jars=['D:/JDBC_Drivers/mainframe_db2/db2jcc4.jar']
) as conn:
    df_current_datetime = pd.read_sql(sql, conn, index_col=None)

In [ ]:
df_current_datetime